<a href="https://colab.research.google.com/github/LEChaney/ML-RayCast-Experiments/blob/master/supervised_learning_4_localization_simpleCNN_can_train_single_or_stacked_frame_V14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/LEChaney/ML-RayCast-Experiments
%cd ML-RayCast-Experiments
%pwd

Cloning into 'ML-RayCast-Experiments'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 518 (delta 18), reused 4 (delta 2), pack-reused 484
Receiving objects: 100% (518/518), 79.89 MiB | 9.67 MiB/s, done.
Resolving deltas: 100% (293/293), done.
/content/ML-RayCast-Experiments


'/content/ML-RayCast-Experiments'

In [0]:
from PIL import Image
def rotate(image_path):
    """
    Rotate the given photo the amount of given degreesk, show it and save it
    @param image_path: The path to the image to edit
    @param degrees_to_rotate: The number of degrees to rotate the image
    @param saved_location: Path to save the cropped image
    """
    degrees_to_rotate = 180
    image_obj = Image.open(image_path)
    rotated_image = image_obj.rotate(degrees_to_rotate)
    rotated_image.save("000.png")
    im = cv2.imread("000.png")
    #plt.imshow(im)
    #plt.show()
    return im
    


In [4]:
import os
os.environ['SDL_VIDEODRIVER']='dummy'
file_directory = os.getcwd()
os.chdir("/content/ML-RayCast-Experiments/Keras-FlappyBird-master/Keras-FlappyBird-master")
print(os.getcwd())

/content/ML-RayCast-Experiments/Keras-FlappyBird-master/Keras-FlappyBird-master


In [5]:
import sys
#sys.path.pop(1)
sys.path.insert(1,"/content/ML-RayCast-Experiments/Keras-FlappyBird-master/Keras-FlappyBird-master/game")
print(sys.path)

['', '/content/ML-RayCast-Experiments/Keras-FlappyBird-master/Keras-FlappyBird-master/game', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython']


In [6]:
from google.colab import drive
drive.mount('/content/gdrive')
!pwd
!ls /content/gdrive/'Shared drives'/'COMPSCI 760'/data

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/ML-RayCast-Experiments/Keras-FlappyBird-master/Keras-FlappyBird-master
'Copy of FlappyBirdData.zip'   data.txt		'The explaination of data.gdoc'
 data_new_1st_million.txt      testData.csv	 trainData.csv
 data_new.txt		       testData.gsheet	 trainData.gsheet


In [7]:
%pip install pygame

     |████████████████████████████████| 11.4MB 2.8MB/s 


In [8]:
sys.path.append("game/")
import wrapped_flappy_bird as game
import numpy as np
import flappy_bird_utils
import cv2
import sys
import random
import pygame
import pygame.surfarray as surfarray
from pygame.locals import *
from itertools import cycle
import matplotlib.pyplot as plt
import PIL.Image
#from cStringIO import StringIO
import IPython.display

import argparse
import skimage as skimage
from skimage import transform, color, exposure
from skimage.transform import rotate
from skimage.viewer import ImageViewer
import json

from collections import deque

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


/usr/local/lib/python3.6/dist-packages/skimage/viewer/__init__.py:6: UserWarning: Viewer requires Qt
  warn('Viewer requires Qt')


In [9]:
GAME = 'bird' # the name of the game being played for log files
CONFIG = 'nothreshold'
ACTIONS = 2 # number of valid actions
GAMMA = 0.99 # decay rate of past observations
OBSERVATION = 2 #3200. # timesteps to observe before training
EXPLORE = 3000000. # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 32 # size of minibatch
FRAME_PER_ACTION = 1
LEARNING_RATE = 1e-4

FPS = 30
SCREENWIDTH  = 288
SCREENHEIGHT = 512

pygame.init()
pygame.display.init()
pygame.display.set_mode((1, 1))
FPSCLOCK = pygame.time.Clock()
SCREEN = pygame.Surface((SCREENWIDTH, SCREENHEIGHT)).convert_alpha()
pygame.display.set_caption('Flappy Bird')

IMAGES, SOUNDS, HITMASKS = flappy_bird_utils.load()
PIPEGAPSIZE = 100 # gap between upper and lower part of pipe
BASEY = SCREENHEIGHT * 0.79

PLAYER_WIDTH = IMAGES['player'][0].get_width()
PLAYER_HEIGHT = IMAGES['player'][0].get_height()
PIPE_WIDTH = IMAGES['pipe'][0].get_width()
PIPE_HEIGHT = IMAGES['pipe'][0].get_height()
BACKGROUND_WIDTH = IMAGES['background'].get_width()

PLAYER_INDEX_GEN = cycle([0, 1, 2, 1])

img_rows , img_cols = 80, 80
#Convert image into Black and white
img_channels = 4 #We stack 4 frames

game_state = game.GameState()
do_nothing = np.zeros(ACTIONS)
do_nothing[0] = 1
game_state.frame_step(do_nothing)

(array([[[0, 0, 0]]], dtype=uint8), 0.1, False)

In [0]:
def get_image_from_coords(data_list, num_channels=1, img_rows=80):
  
    CaseNo = int(float(data_list[0]))
    assert not np.any(np.isnan(np.array(CaseNo)))
    game_state.playerx = int(float(data_list[1]))# + SCREENWIDTH /2)
    game_state.playery = int(float(data_list[2]))# + SCREENWIDTH /2)
    game_state.playerIndex = int(float(data_list[3]))        

    game_state.upperPipes = []
    game_state.lowerPipes = []

    for i in range(4,len(data_list),4):

        newPipe = game.getRandomPipe()
        game_state.upperPipes.append(newPipe[0])
        game_state.upperPipes[int((i-4)/4)]['x'] = int(float(data_list[i]))# + SCREENWIDTH /2)
        game_state.upperPipes[int((i-4)/4)]['y'] = int(float(data_list[i+1]))# + SCREENWIDTH /2)

        game_state.lowerPipes.append(newPipe[1])
        game_state.lowerPipes[int((i-4)/4)]['x'] = int(float(data_list[i+2]))# + SCREENWIDTH /2)
#         print(data_list)
#         print(len(data_list))
#         print(i+3)
        game_state.lowerPipes[int((i-4)/4)]['y'] = int(float(data_list[i+3]))# + SCREENWIDTH /2)

    # draw sprites
    SCREEN.blit(IMAGES['background'], (0,0))

    for uPipe, lPipe in zip(game_state.upperPipes, game_state.lowerPipes):
        SCREEN.blit(IMAGES['pipe'][0], (uPipe['x'], uPipe['y']))
        SCREEN.blit(IMAGES['pipe'][1], (lPipe['x'], lPipe['y']))

    SCREEN.blit(IMAGES['base'], (game_state.basex, BASEY))
    # print score so player overlaps the score
    # showScore(self.score)
    SCREEN.blit(IMAGES['player'][game_state.playerIndex],
                (game_state.playerx, game_state.playery))

    image_data = pygame.surfarray.array3d(SCREEN)
#     print('SCREEN image size: ', image_data.shape)
#     IPython.display.display(PIL.Image.fromarray(image_data))
#     plt.show()
    if num_channels == 1:
        image_data = skimage.color.rgb2gray(image_data) #comment out if using single frame prediction
    
    image_data = skimage.transform.resize(image_data,(img_rows,img_rows))
#     print(image_data.shape)
    if np.any(image_data):
        image_data = skimage.exposure.rescale_intensity(image_data, out_range=(0,255))
    else:
        print('ERROR: completely black image')
#     print(image_data.shape)
    if num_channels == 1:
        image_data = image_data / 255.0
    elif num_channels == 3:
        image_data = preprocess_input(image_data)
#         print('preprocessed data')
    image_data = image_data.reshape(1, image_data.shape[0], image_data.shape[1], num_channels) #1x80x80x1

    label = np.array([game_state.playerx, game_state.playery])
    
#     #print(type(image_data))
#     print("image_data" ,image_data)
#     print("image_data.shape: ", image_data.shape)
#     IPython.display.display(PIL.Image.fromarray(image_data))
#     plt.show()
#     print(image_data.shape)
    return image_data, label

def gen_batch_stacked_frames(file, state, stacked_state_label, bs, D, mode, img_rows=80):
    
    images=[]
    labels=[]

    while len(images) < bs:  
        
        current = file.readline().rstrip()

        #if reach end of the file start again
        if current == "":
            
            file.seek(0)

            current = file.readline().rstrip()
            
            if mode == "eval":
                break
            
            data_list = current.split(',')
            data_list = [x for x in data_list if str(x) != 'nan']
  
            image_data, state_label = get_image_from_coords(data_list, img_rows=img_rows)
            if not np.any(image_data):
                continue
            image_data = image_data.reshape(img_rows, img_rows)
            state = np.stack((image_data, image_data, image_data, image_data), axis=2)
            state = state.reshape(1, state.shape[0], state.shape[1], state.shape[2])  #1*80*80*4
#             state_label = np.array(state_label)
            stacked_state_label = np.concatenate((state_label, state_label, state_label, state_label), axis=0).flatten()
#             print(stacked_state_label)
#             stacked_state_label = stacked_state_label.flatten()
#             print('stacked_state_label: ', stacked_state_label) 
            #state = state.reshape(state.shape[0],)
            
            # store the transition in D
            D.append((state, stacked_state_label))   
            
#             if current[0] == "-":
#                 continue
        else:   
            
            data_list = current.split(',')
#             if len(images) == 10:
#                 print(data_list)
            
            data_list = [x for x in data_list if str(x) != 'nan']
            
#             print('no. in batch: ', len(images))
            
#             if len(images) == 10:
#                 print(data_list)
#                   print(len(data_list))
            
            image_data, state_label = get_image_from_coords(data_list)
            if not np.any(image_data):
                continue
    #             state.append(image_data)
    #             state_labels.append(state_label)
#             print(state.shape)
#             print(image_data.shape)
            state = np.append(image_data, state[:, :, :, :3], axis=3)
#             stacked_state_label = np.append(stacked_state_label, stacked_state_label[:6]).flatten()
#             print(state_label.shape)
#             print(stacked_state_label.shape)
            stacked_state_label = np.concatenate((state_label, stacked_state_label[:6]), axis=0).flatten()
#             print(stacked_state_label)
#             stacked_state_label = stacked_state_label.flatten()
#             print('stacked_state_label.shape: ', stacked_state_label.shape) 
            #state = np.array(state)
            #state = state.reshape(state.shape[0],)
        
            # store the transition in D
            D.append((state, stacked_state_label))
        
        if len(D) > REPLAY_MEMORY:
            D.popleft()
#         print(len(D))

        # training with experience replay
        miniBatch = random.sample(D,1)
        gen_state, gen_label = zip(*miniBatch)
#         print('gen_state: ', gen_state[0])
#         print('gen_label: ', gen_label[0])
        images.append(gen_state[0])
#         print(gen_label[0])
        labels.append(gen_label[0])
#         print('len(labels): ', len(labels))
    
    images = np.array(images)
#     print(images.shape)
    images = images.reshape(bs, img_rows, img_rows, 4)
#     print(images.shape)
#     print('labels: ', labels)
    labels = np.array(labels)
#     print('labels: ', labels)
#     print('labels.shape: ', labels.shape)
    labels = labels.reshape(bs, 8)
#     print('labels.shape: ', labels.shape)
    # if the data augmentation object is not None, apply it
    if aug is not None:
      (images, labels) = next(aug.flow(images, labels, batch_size=bs))
    
    return images, labels, state, stacked_state_label
  
def gen_batch_single_frame(file, bs, D, mode, num_channels, img_rows=80):
    
    images=[]
    labels=[]

    while len(images) < bs:  
        
        current = file.readline().rstrip()

        #if reach end of the file start again
        if current == "":
            
            file.seek(0)
            current = file.readline().rstrip()
            
            if mode == "eval":
                break
            
        data_list = current.split(',')
#         if len(images) == 10:
#             print(data_list)

        data_list = [x for x in data_list if str(x) != 'nan']

#         print('no. in batch: ', len(images))

#         if len(images) == 10:
#             print(data_list)
#               print(len(data_list))

        image_data, state_label = get_image_from_coords(data_list, num_channels=num_channels, img_rows=img_rows)
        if not np.any(image_data):
            continue
#         print(image_data.shape)
        # store the transition in D
        if D is not None:
            D.append((state, state_label))
            if len(D) > REPLAY_MEMORY:
                D.popleft()
#             print(len(D))
        
            # training with experience replay
            miniBatch = random.sample(D,1)
            gen_state, gen_label = zip(*miniBatch)
            assert not np.any(np.isnan(gen_state))
            assert not np.any(np.isnan(gen_label))
            images.append(gen_state)
            labels.append(gen_label)
        else:
            images.append(image_data)
            labels.append(state_label)
    
    images = np.array(images)
#     print(images.shape)
    images = images.reshape(bs, img_rows, img_rows, num_channels)
#     print(images.shape)
    labels = np.array(labels)
#     print(labels.shape)
    labels = labels.reshape(bs, 2)
#     print(labels.shape)
    # if the data augmentation object is not None, apply it
    if aug is not None:
      (images, labels) = next(aug.flow(images, labels, batch_size=bs))
    
    return images, labels

In [0]:

'''import numpy as np
import sys
import random
import pygame
import flappy_bird_utils
import pygame.surfarray as surfarray
from pygame.locals import *
from itertools import cycle'''

def csv_image_generator(filepath, bs, mode="train", aug=None, use_4_frames=False, useExperienceReplay=False, num_channels=1, img_rows=80):

    file = open(filepath,"r")
    
    maxCount = 100
    count = maxCount
       
    # initialize state
    current = file.readline().rstrip()
  
    data_list = current.split(',')

#     print(data_list)
    data_list = [x for x in data_list if str(x) != 'nan']          
#     print(data_list)

    image_data, state_label = get_image_from_coords(data_list, num_channels=num_channels, img_rows=img_rows)
#     print(image_data.shape)
#     print(type(image_data))
#     print("image_data" ,image_data)
#     print("image_data.shape: ", image_data.shape)
#     IPython.display.display(PIL.Image.fromarray(image_data))
#     plt.show()
    if num_channels == 3:
        image_data = image_data.reshape(img_rows, img_rows, num_channels)
    elif num_channels == 1:
        image_data = image_data.reshape(img_rows, img_rows)
#     print(image_data.shape)
  
    if use_4_frames:

        state = np.stack((image_data, image_data, image_data, image_data), axis=2)
  #         print(state.shape)
        state = state.reshape(1, state.shape[0], state.shape[1], state.shape[2])  #1*img_rows*img_rows*4
        stacked_state_label = np.concatenate((state_label, state_label, state_label, state_label), axis=0).flatten()
#         print('stacked_state_label first: ', stacked_state_label)
#         stacked_state_label = stacked_state_label.flatten()
        print('stacked_state_label first flattened: ', stacked_state_label)    

        # store the previous observations in replay memory
        D = deque()
        D.append((state, stacked_state_label)) 

        while True:

  #             count -= 1
  #             if count > 1:
  #                 continue
  #             if count == 0:
  #                 break;
  #             print('count: ', count)

            images, labels, state, stacked_state_label = gen_batch_stacked_frames(file, state, stacked_state_label, bs, D, mode, img_rows)

            # yield the batch to the calling function
  #             print(images.shape)
  #             print(labels.shape)

            yield (images, labels)    

    elif useExperienceReplay:    

        # store the previous observations in replay memory
        D = deque()
        D.append((image_data, state_label)) 

        while True:
          
#             count -= 1
#             if count > 1:
#                 continue
#             if count == 0:
#                 break;
#             print('count: ', count)
            
            images, labels = gen_batch_single_frame(file, bs, D, mode, num_channels, img_rows=img_rows)
        
            # yield the batch to the calling function
#             print(images.shape)
#             print(labels.shape)
            
            yield (images, labels)   
    
    else:
      
        while True:
          
#             count -= 1
#             if count > 1:
#                 continue
#             if count == 0:
#                 break;
#             print('count: ', count)
            
            images, labels = gen_batch_single_frame(file, bs, D=None, mode=mode, num_channels=num_channels, img_rows=img_rows)
        
            # yield the batch to the calling function
#             print(images.shape)
#             print(labels.shape)
            
            yield (images, labels)   

In [0]:
# file= "data.csv" #file path or a file in the current folder
# csv_image_generator(file, 8, mode="train", aug=None)

<generator object csv_image_generator at 0x7f6805691990>

In [12]:
!pip install Keras-CoordConv
# !pip install pyimagesearch

  Created wheel for Keras-CoordConv: filename=Keras_CoordConv-0.6-cp36-none-any.whl size=7578 sha256=60dc3b290af30f6bf067fb77ca54dda381e5b71d8bcef913146f04a8ae0bf1b3
  Stored in directory: /root/.cache/pip/wheels/38/32/98/8eeb612bb1bee30a9ee99ff406da961bcfef37fc7cf14f4120
Successfully built Keras-CoordConv


In [0]:
!rm trainData.csv
!rm testData.csv
!ls

rm: cannot remove 'trainData.csv': No such file or directory
rm: cannot remove 'testData.csv': No such file or directory
animation1.gif	game			model.json	       qlearn_ray.py
assets		__init__.py		qlearn_glimpse.py      README.md
data.csv	ML-RayCast-Experiments	qlearn_loc_and_ray.py
data.txt	model.h5		qlearn.py


In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

# USAGE
# python train.py

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
import sys
from __future__ import print_function

from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam
from keras.initializers import normal, identity
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import Input
from keras import Model
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from keras.layers import Conv2D, Reshape
from keras.utils import Sequence
from keras import backend
from keras.backend import epsilon
from keras.callbacks import Callback
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

import random
import numpy as np
import math
from Keras_CoordConv.coord import CoordinateChannel2D
# from coord import CoordinateChannel2D

from sklearn.model_selection import train_test_split

img_rows , img_cols = 80, 80
#Convert image into Black and white

use_single_frame = True
use_4_frames = False #If We stack 4 frames for model input
use_rgb = True

if use_single_frame == True:
    if use_rgb:
        img_channels = 3 #rgb single frame for model input
    else:
        img_channels = 1 #greyscale single frame for model input
    out_shape = 2 #2 ouptut neurons for single x, y player coordinate for output
else:
    img_channels = 4 #stack 4 frames for model input
    out_shape = 8 #8 output neurons for 4 pairs of x, y player coordinate for output
    
usingSimpleCNN = 0
usingSimpleCoordConv = 0
usingMobileNetV2Localizer = 1

if usingMobileNetV2Localizer == 1:
    backend.set_image_data_format('channels_last')
    img_rows, img_cols = 96, 96
# print('img_rows: ', img_rows)

useExperienceReplay = False

MAX_NUM_SAMPLES = 100000

np.random.seed(2019)

def buildSimpleCNN():
    print("Now we build the model")
    model = Sequential()
    model.add(Convolution2D(32, 8, 8, subsample=(4,4), border_mode='same', input_shape=(img_cols,img_rows,img_channels)))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 4, 4, subsample=(2,2), border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, 3, subsample=(1,1), border_mode='same'))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(out_shape))
   
    adam = Adam(lr=1e-3)
    model.compile(loss='mse',optimizer=adam)
    print("We finish building the model")
    return model

def create_COORDCONV_Simple_CNN():
    print("Now we build the model")
    ip = Input(shape=(img_rows,img_cols,img_channels))
    x = CoordinateChannel2D()(ip)
    x = Convolution2D(32, 8, 8, subsample=(4,4), border_mode='same')
    x = Activation('relu')
    x = CoordinateChannel2D()(x)
    x = Convolution2D(64, 4, 4, subsample=(2,2), border_mode='same')
    x = Activation('relu')
    x = CoordinateChannel2D()(x)
    x = Convolution2D(64, 3, 3, subsample=(1,1), border_mode='same')
    x = Activation('relu')
    x = Flatten()
    x = Dense(512)
    x = Activation('relu')
    x = Dense(out_shape)
    model = keras.Model(inputs=ip, outputs=x)
    adam = Adam(lr=1e-6)
    model.compile(loss='mse',optimizer=adam)
    return model

"""
# prior to first conv
ip = Input(shape=(64, 64, 2))
x = CoordinateChannel2D()(ip)
x = Conv2D(...)(x)  # This defines the `CoordConv` from the paper.
...
x = CoordinateChannel2D(use_radius=True)(x)
x = Conv2D(...)(x)  # This adds the 3rd channel for the radius.
"""
def create_MobileNetV2_model(trainable=False):
    input_tensor = Input(shape=(img_rows,img_cols,img_channels)) 
    model = MobileNetV2(input_tensor=input_tensor, weights='imagenet', input_shape=(img_rows,img_cols,img_channels), include_top=False)

    # to freeze layers
    for layer in model.layers:
        layer.trainable = trainable

    x = model.layers[-1].output
    x = Conv2D(out_shape, kernel_size=3, name="coords")(x)
    x = Reshape((out_shape,))(x)

    model = Model(inputs=model.input, outputs=x)
    adam = Adam(lr=1e-3)
    model.compile(loss='mse', optimizer=adam)

    return model

def get_train_test_split(data_path):
    file = open(data_path+'data_new_1st_million.txt','r')
    dataList = [line.rstrip().split() for line in file]
    dataList = [[int(float(x)) for x in line] for line in dataList]
#     print(dataList[0])
    data = np.empty((len(dataList),len(max(dataList,key=len))))
    data[:,:] = np.nan
    for i in range(len(dataList)):
        data[i,:len(dataList[i])] = np.array(dataList[i])
        assert not np.any(np.isnan(data[i,:len(dataList[i])]))
#     print(data[0,:])
    file.close()
#     data = np.genfromtxt(data_path+'data_new_1st_million.txt', delimiter=' ')
    if use_single_frame and not useExperienceReplay:
        data = np.take(data,np.random.rand(data.shape[0]).argsort(),axis=0,out=data)
        print('shuffled data')
#         data = np.random.shuffle(data)
    data = data[:MAX_NUM_SAMPLES,:]
    traintestsplit = np.split(data, [int(4*data.shape[0]/5)])
    train = traintestsplit[0]
    test = traintestsplit[1]
#     print(train[108])
    return train, test

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))

    
history = LossHistory()    

# initialize the paths to our training and testing CSV files
#TRAIN_CSV = "flowers17_training.csv"
#TEST_CSV = "flowers17_testing.csv"
data_path='/content/gdrive/Shared drives/COMPSCI 760/data/'
TRAIN, TEST = get_train_test_split(data_path)

# initialize the number of epochs to train for and batch size
NUM_EPOCHS = 1#5#75
BS = 32

# initialize the total number of training and testing image
NUM_TRAIN_IMAGES = 0
NUM_TEST_IMAGES = 0

# open the training CSV file, then initialize the unique set of class
# labels in the dataset along with the testing labels
#f = open(TRAIN_CSV, "r")
labels = set()
testLabels = []

'''
# loop over all rows of the CSV file
for row in TRAIN:
	# extract the class label, update the labels list, and increment
	# the total number of training images
	label = line.strip().split(",")[0]
	labels.add(label)
	NUM_TRAIN_IMAGES += 1

# close the training CSV file and open the testing CSV file
f.close()
f = open(TEST_CSV, "r")

# loop over the lines in the testing file
for row in TEST:
	# extract the class label, update the test labels list, and
	# increment the total number of testing images
	label = line.strip().split(",")[0]
	testLabels.append(label)
	NUM_TEST_IMAGES += 1

# close the testing CSV file
f.close()
'''

trainLabels = TRAIN[:,1:3]
# trainLabels[0] /= SCREENWIDTH
# trainLabels[1] /= SCREENHEIGHT
trainData = TRAIN[:,:]
np.savetxt(data_path+'trainData.csv', trainData, delimiter=',')
NUM_TRAIN_IMAGES = trainData.shape[0]

print('num train images: ', NUM_TRAIN_IMAGES)

# file = open('trainData.csv', "r")

# for i in range(101):
#     current = file.readline().rstrip()
# data_list = current.split(',')
# print(data_list)
# data_list = [x for x in data_list if str(x) != 'nan']          
# print(data_list)
# image_data, state_label = get_image_from_coords(data_list)
# file.close()
# # print(type(image_data))
# # print("image_data" ,image_data)

# image_data = image_data.reshape(80,80)
# print("image_data.shape: ", image_data.shape)

# IPython.display.display(PIL.Image.fromarray(image_data))
# plt.show()


testLabels = TEST[:,1:3]
# testLabels[0] /= SCREENWIDTH
# testLabels[1] /= SCREENHEIGHT
testData = TEST[:,:]
np.savetxt(data_path+'testData.csv', testData, delimiter=',')
NUM_TEST_IMAGES = testData.shape[0]

print('num test images: ', NUM_TEST_IMAGES)

# create the label binarizer for one-hot encoding labels, then encode
# the testing labels
# lb = LabelBinarizer()
# lb.fit(list(trainLabels))
# testLabels = lb.transform(testLabels)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(width_shift_range=0.2, height_shift_range=0.2, fill_mode="nearest")

# initialize both the training and testing image generators
trainGen = csv_image_generator(data_path+'trainData.csv', BS, #lb,
    mode="train", aug=aug,
    use_4_frames=use_4_frames, useExperienceReplay=useExperienceReplay,
    num_channels=img_channels, img_rows=img_rows
  )

testGen = csv_image_generator(data_path+'testData.csv', BS, #lb,
    mode="train", aug=None,
    use_4_frames=use_4_frames, useExperienceReplay=useExperienceReplay,
    num_channels=img_channels, img_rows=img_rows
  )

# initialize our Keras model and compile it
if usingSimpleCNN:
    model = buildSimpleCNN()
elif usingSimpleCoordConv:
    model = create_COORDCONV_Simple_CNN()    
elif usingMobileNetV2Localizer:
    model = create_MobileNetV2_model()

# train the network
print("[INFO] training w/ generator...")
H = model.fit_generator(
	trainGen,
	steps_per_epoch=math.ceil(NUM_TRAIN_IMAGES / BS),
	validation_data=testGen,
	validation_steps=math.ceil(NUM_TEST_IMAGES / BS),
	epochs=NUM_EPOCHS,
  callbacks=[history]
)

model.save(data_path+'model_'+str(NUM_TRAIN_IMAGES)+'_train_images_'+str(NUM_TEST_IMAGES)+'_val_images_'+str(NUM_EPOCHS)+'_epochs.h5')

# plot the training loss and accuracy
N = int(NUM_EPOCHS*math.ceil(NUM_TRAIN_IMAGES / BS))
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history.losses, label="train_loss")
plt.plot(np.arange(0, N), history.val_losses, label="val_loss")
# plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
# plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Loss on Dataset")
plt.xlabel("Iteration #")
plt.ylabel("MSE Loss")#/Accuracy")
plt.legend(loc="lower left")
plt.savefig(data_path+'Learning_Curve_'+str(NUM_TRAIN_IMAGES)+'_train_images_'+str(NUM_TEST_IMAGES)+'_val_images_'+str(NUM_EPOCHS)+'_epochs.png')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
shuffled data
num train images:  80000
num test images:  20000











9412608/9406464 [==============================] - 2s 0us/step

[INFO] training w/ generator...


Epoch 1/1
2499/2500 [============================>.] - ETA: 1s - loss: 880.4428ERROR: completely black image
ERROR: completely black image
2500/2500 [==============================] - 5727s 2s/step - loss: 880.3401 - val_loss: 1129.4056


ValueError: ignored

In [0]:
print((history.losses).shape)

In [0]:
# re-initialize our testing data generator, this time for evaluating
testGen = csv_image_generator(data_path+'testData.csv', BS, #lb,
    mode="eval", aug=None,
    use_4_frames=use_4_frames, useExperienceReplay=useExperienceReplay,
    num_channels=img_channels, img_rows=img_rows
  )

# make predictions on the testing images, finding the index of the
# label with the corresponding largest predicted probability
predIdxs = model.predict_generator(testGen,
	steps=(NUM_TEST_IMAGES // BS) + 1)
# predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print("[INFO] evaluating network...")
# print(classification_report(testLabels.argmax(axis=1), predIdxs,
# 	target_names=lb.classes_))

# plot the training loss and accuracy
N = int(NUM_EPOCHS*math.ceil(NUM_TRAIN_IMAGES / BS))
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history.losses, label="train_loss")
plt.plot(np.arange(0, N), history.val_losses, label="val_loss")
# plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
# plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Loss on Dataset")
plt.xlabel("Iteration #")
plt.ylabel("MSE Loss")#/Accuracy")
plt.legend(loc="lower left")
plt.savefig(data_path+'Learning_Curve_'+str(NUM_TRAIN_IMAGES)+'_train_images_'+str(NUM_TEST_IMAGES)+'_val_images_'+str(NUM_EPOCHS)+'_epochs.png')

In [0]:
!ls '/content/gdrive/Shared drives/COMPSCI 760/data/'

'Copy of FlappyBirdData.zip'   data.txt		'The explaination of data.gdoc'
 data_new_1st_million.txt      testData.csv	 trainData.csv
 data_new.txt		       testData.gsheet	 trainData.gsheet


In [0]:
from keras import backend
print(backend.image_data_format())

channels_last
